In [37]:
import pandas as pd
import sys
sys.path.append("../helper_functions")
from web3 import Web3
from datetime import datetime, timezone
sys.path.pop()
import re

In [38]:
def clean_string(raw_str: str) -> str:
    """
    Remove all control characters (ASCII 0–31, plus 127),
    then strip leading/trailing whitespace.
    """
    return re.sub(r'[\x00-\x1F\x7F]+', '', raw_str).strip()


def get_token_metadata(token_contract_address, rpc, method_name_list):
    """
    Aggregates multiple contract calls into one Python function call.
    (Still multiple calls to your Ethereum node, but simpler to manage in code.)
    """
    w3_conn = Web3(Web3.HTTPProvider(rpc))
    token_contract_address = token_contract_address.lower() 
    
    block_timestamp = w3_conn.eth.get_block('latest').timestamp
    block_number = w3_conn.eth.get_block('latest').number
    # Convert the UNIX timestamp to a human-readable format
    block_datetime = datetime.fromtimestamp(block_timestamp, tz=timezone.utc)
    block_time = block_datetime.strftime('%Y-%m-%d %H:%M:%S')

    method_ids = [Web3.keccak(text=f"{method_name}()")[:4].hex() for method_name in METHOD_NAME]
    
    # create a loop to get the response for each method
    response_map = {}

    for method_id, method_name in zip(method_ids, method_name_list):
        response = w3_conn.eth.call({
            'to': token_contract_address,
            'data': method_id
        })
        response_map[method_name] = response


    # return the response for each method
    
    decimals = Web3.to_int(hexstr=response_map["decimals"].hex())
    symbol = clean_string(Web3.to_text(hexstr=response_map["symbol"].hex()))
    name = clean_string(Web3.to_text(hexstr=response_map["name"].hex()))
    total_supply = Web3.to_int(hexstr=response_map["totalSupply"].hex())
    
    return {
        "block_time": block_time,
        "block_number": block_number,
        "block_timestamp": block_timestamp,
        "contract_address": token_contract_address,
        "decimals": decimals,
        "symbol": symbol,
        "name": name,
        "total_supply": total_supply
    }

In [39]:
chains_rpcs = pd.read_csv('../../op_chains_tracking/outputs/chain_metadata.csv', na_filter=False)
chains_rpcs = chains_rpcs[~(chains_rpcs['rpc_url'] == '')]

rpc =chains_rpcs[chains_rpcs['chain_name'] == 'op']["rpc_url"]
RPC = rpc.iloc[0] 

CONTRACT_ADDRESS = "0x4200000000000000000000000000000000000006"
METHOD_NAME = ["decimals", "symbol", "name", "totalSupply"]


In [40]:
response = get_token_metadata(CONTRACT_ADDRESS, RPC, METHOD_NAME)
response

{'block_time': '2025-02-12 20:56:55',
 'block_number': 131897519,
 'block_timestamp': 1739393815,
 'contract_address': '0x4200000000000000000000000000000000000006',
 'decimals': 18,
 'symbol': 'WETH',
 'name': 'Wrapped Ether',
 'total_supply': 38751362421522965331729}